In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import sys
sys.path.append("/home/ubuntu/Notebooks/annsa/")

import annsa as an
import numpy as np

/home/ubuntu/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [73]:
an.isotopes[0:-3]

['Am241',
 'Ba133',
 'Co57',
 'Co60',
 'Cs137',
 'Cr51',
 'Eu152',
 'Ga67',
 'I123',
 'I125',
 'I131',
 'In111',
 'Ir192',
 'U238',
 'Lu177m',
 'Mo99',
 'Np237',
 'Pd103',
 'Pu239',
 'Pu240',
 'Ra226',
 'Se75',
 'Sm153',
 'Tc99m',
 'Xe133',
 'Tl201',
 'Tl204',
 'U233',
 'U235']

In [76]:
print an.isotopes[0:9]
print an.isotopes[9:18]
print an.isotopes[18:-3]

['Am241', 'Ba133', 'Co57', 'Co60', 'Cs137', 'Cr51', 'Eu152', 'Ga67', 'I123']
['I125', 'I131', 'In111', 'Ir192', 'U238', 'Lu177m', 'Mo99', 'Np237', 'Pd103']
['Pu239', 'Pu240', 'Ra226', 'Se75', 'Sm153', 'Tc99m', 'Xe133', 'Tl201', 'Tl204', 'U233', 'U235']


## Load template spectra

In [23]:
def load_template_spectra_from_folder(parent_folder,spectrum_identifier):
    '''
    inputs: partent_folder, spectrum_identifier
    output: dictionary containing all template spectra from a folder.
    
    Load template spectrum data into a dictionary. This allows templates from 
    different folders to be loaded into different dictionaries.
    
    '''
    
    
    temp_dict = {}
    
    def normalize_spectrum(ID):
        temp_spectrum = an.read_spectrum(parent_folder + ID + spectrum_identifier)
        if np.max(temp_spectrum)==0:
            print ID + ' Contains no values'
        return temp_spectrum/np.max(temp_spectrum)
    
    for i in range(len(an.isotopes)-3):
#        if an.isotopes[i]=='I125':
#            continue
        temp_dict[an.isotopes[i]] = normalize_spectrum(an.isotopes_sources_GADRAS_ID[i]) # 0 
    

    return temp_dict

In [24]:
spectral_templates={}

spectrum_identifier = "_10uC_spectrum.spe"

spectral_templates['noshield']=load_template_spectra_from_folder("templates/no-shielding/",
                                                                   spectrum_identifier)

spectral_templates['aluminum20pct']=load_template_spectra_from_folder("templates/aluminum-20pct/",
                                                                        spectrum_identifier)
spectral_templates['aluminum40pct']=load_template_spectra_from_folder("templates/aluminum-40pct/",
                                                                        spectrum_identifier)
spectral_templates['aluminum60pct']=load_template_spectra_from_folder("templates/aluminum-60pct/",
                                                                        spectrum_identifier)
spectral_templates['aluminum80pct']=load_template_spectra_from_folder("templates/aluminum-80pct/",
                                                                        spectrum_identifier)

spectral_templates['iron20pct']=load_template_spectra_from_folder("templates/iron-20pct/",
                                                                        spectrum_identifier)
spectral_templates['iron40pct']=load_template_spectra_from_folder("templates/iron-40pct/",
                                                                        spectrum_identifier)
spectral_templates['iron60pct']=load_template_spectra_from_folder("templates/iron-60pct/",
                                                                        spectrum_identifier)
spectral_templates['iron80pct']=load_template_spectra_from_folder("templates/iron-80pct/",
                                                                        spectrum_identifier)


spectral_templates['lead20pct']=load_template_spectra_from_folder("templates/lead-20pct/",
                                                                        spectrum_identifier)
spectral_templates['lead40pct']=load_template_spectra_from_folder("templates/lead-40pct/",
                                                                        spectrum_identifier)
spectral_templates['lead60pct']=load_template_spectra_from_folder("templates/lead-60pct/",
                                                                        spectrum_identifier)
spectral_templates['lead80pct']=load_template_spectra_from_folder("templates/lead-80pct/",
                                                                        spectrum_identifier)

125I Contains no values
125I Contains no values
125I Contains no values
125I Contains no values
125I Contains no values
125I Contains no values
125I Contains no values


/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: RuntimeWarning: invalid value encountered in divide


In [25]:
background_locations=['albuquerque',
                      'chicago',
                      'denver',
                      'losalamos',
                      'miami',
                      'newyork',
                      'sanfrancisco']

In [26]:
spectral_templates['background']={}

def normalize_spectrum(location):
        temp_spectrum = an.read_spectrum('./templates/background/background-'+location+'.spe')
        if np.max(temp_spectrum)==0:
            print ID + ' Contains no values'
        return temp_spectrum/np.max(temp_spectrum)

for location in background_locations:
    spectral_templates['background'][location]=normalize_spectrum(location)

### Define dataset parameters

In [6]:
# Assuming 60cps background count rate on detector

def generate_random_counts_on_detector(integration_time_setting):
    
    integration_time = np.random.uniform(integration_time_ranges[integration_time_setting][0],
                                         integration_time_ranges[integration_time_setting][1])

    count_rate_on_detector = np.random.uniform(10,120)
    counts_on_detector = int(count_rate_on_detector*integration_time)

    return counts_on_detector

In [7]:
spectral_template_settings=['noshield',
                           'aluminum20pct',
                           'aluminum40pct',
                           'aluminum60pct',
                           'aluminum80pct',
                           'iron20pct',
                           'iron40pct',
                           'iron60pct',
                           'iron80pct',
                           'lead20pct',
                           'lead40pct',
                           'lead60pct',
                           'lead80pct',]

isotope_list=an.isotopes[:-3]

In [66]:
integration_time_division=10
signal_to_background_division=5
calibration_division=5

integration_times=np.logspace(np.log10(10),np.log10(3600),integration_time_division)
signal_to_backgrounds=np.logspace(np.log10(0.25),np.log10(4),signal_to_background_division)
calibrations=np.linspace(0.952,1.428,calibration_division)

#print '\n\ntotal samples: ' + str(integration_time_division*signal_to_background_division*calibration_division)

In [41]:
all_source_spectra=[]
all_keys=[]
LLD=10
background_cps=85.
total_spectra=0


for isotope in isotope_list:
    for spectral_template_setting in spectral_template_settings:
        for integration_time in integration_times:
            for signal_to_background in signal_to_backgrounds:
                for calibration in calibrations:
                    isotopes_per_setting=1
                    # Simulate extra unshielded spectra to avoid training set imbalance 
                    if spectral_template_setting=='noshield':
                        isotopes_per_setting*=12
                    for _ in range(isotopes_per_setting):

                        
                        # I125 emits a maximum gamma-ray energy of 30keV. 
                        # This will either be completely attenuated by any shielding, or produce insignificatn Compton Continuum.
                        # For this reason shielded I125 is removed as a class
                        if isotope=='I125' and spectral_template_setting!='noshield':
                            continue
                        
                        # Simulate source
                        source_template=spectral_templates[spectral_template_setting][isotope]
                        source_template=an.zoom_spectrum(source_template,calibration)
                        source_template[0:LLD]=0
                        
                        sampled_spectrum=an.sample_spectrum(source_template,integration_time*background_cps*signal_to_background)
                        flag=0
                        while np.sum(np.isnan(sampled_spectrum))!=0:
                            sampled_spectrum=an.sample_spectrum(source_template,
                                           generate_random_counts_on_detector(integration_time))
                            flag+=1
                            if flag==1:
                                print 'spectral template'+spectral_templates[spectral_template_setting]+'contains NaN'
                                break
                       
                        all_source_spectra.append(sampled_spectrum)
                        
                        isotope_key=isotope
                        
                        if spectral_template_setting!='noshield':
                            isotope_key+='_shielded'
                        all_keys.append(isotope_key)
                        
                        print ('\1b[2k\r'),    
                        print('Isotope %s, template %s, %s total spectra simulated' %(isotope,
                                                                                      spectral_template_setting,
                                                                                      total_spectra)),
                        
                        if total_spectra%100==0:
                            np.save('CVT-oct-source-spectra-test-v1',all_source_spectra)
                            np.save('CVT-oct-keys-test-v1',all_keys)
                            
np.save('CVT-oct-source-spectra-test-v1',all_source_spectra)
np.save('CVT-oct-keys-test-v1',all_keys)

Isotope U235, template lead80pct, 171000 total spectra simulated
